# Coursework 1
## Part 3. Detecting problematic modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
DATA_PATH = "./Data"

In [3]:
df_dict = {}

for file_ in os.listdir(DATA_PATH):
    df_dict[file_] = pd.read_csv(os.path.join(DATA_PATH, file_))
    
df_dict["progresses.csv"].columns = ["progress_id", "student_id", "course_id"]

df_dict["students.csv"].columns = ['id_', 'student_id', 'city', 'birthday']
df_dict["students.csv"]["birthday"] = pd.to_datetime(df_dict["students.csv"]["birthday"], errors="coerce")

df_dict["courses.csv"].columns = ["index", "course_id", "course_title", "course_field"]
df_dict["courses.csv"].set_index("index", inplace=True)

dataset = pd.merge(left=df_dict["progresses.csv"],
                   right=df_dict["progress_phases.csv"],
                   on="progress_id",
                   how="outer")

dataset = pd.merge(left=dataset,
                   right=df_dict["courses.csv"],
                   on="course_id",
                   how="outer")

dataset = pd.merge(left=dataset,
                   right=df_dict["course_contents.csv"],
                   on=["course_id", "lesson_number", "module_number"],
                   how="outer")

dataset["start_date"] = (
    pd.to_datetime(dataset["start_date"], errors="coerce", utc=False)
    .dt.tz_localize(None)
    )

dataset["finish_date"] = (
    pd.to_datetime(dataset["finish_date"], errors="coerce", utc=False)
    .dt.tz_localize(None)
    )

dset_students = df_dict["students.csv"].copy(deep=True)
dset_students["age_years"] = (np.datetime64(str(2022)) - dset_students["birthday"]).astype("timedelta64[Y]")
#dset_students["student_age"] = dset_students["student_age"].astype("timedelta64[Y]")

print("Main dataset:", dataset.dtypes, dataset.columns, sep="\n", end="\n\n")
print("Student dataset:", dset_students.dtypes, dset_students.columns, sep="\n")

Main dataset:
progress_id              object
student_id               object
course_id                object
module_number           float64
lesson_number           float64
status                   object
start_date       datetime64[ns]
finish_date      datetime64[ns]
course_title             object
course_field             object
module_title             object
lesson_title             object
lesson_token             object
is_video                 object
is_homework              object
dtype: object
Index(['progress_id', 'student_id', 'course_id', 'module_number',
       'lesson_number', 'status', 'start_date', 'finish_date', 'course_title',
       'course_field', 'module_title', 'lesson_title', 'lesson_token',
       'is_video', 'is_homework'],
      dtype='object')

Student dataset:
id_                    int64
student_id            object
city                  object
birthday      datetime64[ns]
age_years            float64
dtype: object
Index(['id_', 'student_id', 'city', 'birth

### Questions

* how many unfinished modules?
* duration of each lesson
* duration of spaces between lessons?
* total module duration divided by number of homeworks
* difference between mean and median time

### Making a course-module id to avoid nested groupby
because module is being identified with just a number

by joining course_id and module_number

In [4]:
dataset["course-module_id"] = (
    dataset["course_id"] +
    "_" +
    dataset["module_number"].astype("str")
)

dataset["course-module_id"].sample(n=10)

91378      6e4837f0a65c68efbfdd0594d6774701_1.0
187629     dad6f6ffc086caa89e2f40c28a9c7490_6.0
141121     abce125a877c2196a3bc7bfbc11b5fc5_7.0
186647    dad6f6ffc086caa89e2f40c28a9c7490_14.0
329066     943306102e5b067d08a29094f37b8193_1.0
9900       0770b1b039964228294f1f34b29fc2c1_9.0
161523     480d25ad36878fce09e7aada316ba1a6_5.0
293005     e3a0df907589d56692b5cb561fd90d87_3.0
138964     abce125a877c2196a3bc7bfbc11b5fc5_2.0
185354     dad6f6ffc086caa89e2f40c28a9c7490_3.0
Name: course-module_id, dtype: object

How many unique modules?

In [5]:
dataset["course-module_id"].nunique()

230

### Time to complete module - minimum

modules having 1 homework: 1 student = 1 progress ?

2 or more homeworks: 1 student = many progresses ?

progress / student ratio

In [6]:
temp_list = []

for module_id_, module_gr_ in dataset[dataset["is_homework"] == True].groupby("course-module_id"):
    
    if module_gr_["lesson_number"].nunique() > 1:
    
        temp_list.append(module_gr_["progress_id"].nunique() / module_gr_["student_id"].nunique())


pd.Series(temp_list)

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
dtype: float64

something's wrong

In [7]:
for module_id_, module_gr_ in dataset[dataset["is_homework"] == True].groupby("course-module_id"):
    
    if module_gr_["lesson_number"].nunique() > 1:
        
        print(dataset[dataset["course-module_id"] == module_id_]["course_title"].unique(),
              module_gr_["lesson_number"].unique(),
              module_id_)

['Java-разработчик c нуля'] [27. 21. 13. 17.  6. 25.] 943306102e5b067d08a29094f37b8193_4.0
['Java-разработчик c нуля'] [ 7. 13. 11.  5.  9.] 943306102e5b067d08a29094f37b8193_5.0
['Java-разработчик c нуля'] [13.  4.  6.  2.] 943306102e5b067d08a29094f37b8193_6.0
['Java-разработчик c нуля'] [8. 2.] 943306102e5b067d08a29094f37b8193_7.0
['Java-разработчик c нуля'] [ 8.  4. 11.] 943306102e5b067d08a29094f37b8193_8.0
['Java-разработчик c нуля'] [ 5.  7.  9. 12. 14.] 943306102e5b067d08a29094f37b8193_9.0
['Веб-дизайн PRO 2.0'] [3. 5.] c9fcb746d51e41bc5a217002d081454b_4.0
['UX-дизайн'] [7. 8.] dad6f6ffc086caa89e2f40c28a9c7490_9.0


Вопрос: найти общие для уроков одного модуля student_id

In [8]:
check = dataset[
    (dataset["is_homework"] == True) &
    (dataset["course-module_id"] == "943306102e5b067d08a29094f37b8193_4.0")].groupby("lesson_number")

check.count()

,progress_id,student_id,course_id,module_number,status,start_date,finish_date,course_title,course_field,module_title,lesson_title,lesson_token,is_video,is_homework,course-module_id
lesson_number,,,,,,,,,,,,,,,
6.0,162,162,162,162,162,162,151,162,162,162,162,162,162,162,162
13.0,147,147,147,147,147,147,142,147,147,147,147,147,147,147,147
17.0,141,141,141,141,141,141,131,141,141,141,141,141,141,141,141
21.0,133,133,133,133,133,133,122,133,133,133,133,133,133,133,133
25.0,126,126,126,126,126,126,113,126,126,126,126,126,126,126,126
27.0,115,115,115,115,115,115,110,115,115,115,115,115,115,115,115


number of students decreasing along hw completion in module

let's find a student, who did completed all homeworks in module - take one from the last group (27 lesson)

In [9]:
check.get_group(list(check.groups)[-1]).head(1)

,progress_id,student_id,course_id,module_number,lesson_number,status,start_date,finish_date,course_title,course_field,module_title,lesson_title,lesson_token,is_video,is_homework,course-module_id
334902,d0cecd82f603e457eb1142def0fa3104,47c3c0e20c441a4fa021ca4bd8044e27,943306102e5b067d08a29094f37b8193,4.0,27.0,done,2019-04-11 10:56:03.684644,2019-04-14 03:12:23.012366,Java-разработчик c нуля,Development,"Числа, строки и даты",Домашняя работа 4.6,ed373787-d237-4fb3-8248-82595b0dde56,False,True,943306102e5b067d08a29094f37b8193_4.0


In [10]:
dataset[
    (dataset["student_id"] == "47c3c0e20c441a4fa021ca4bd8044e27") & 
    (dataset["is_homework"] == True) & 
    (dataset["course-module_id"] == "943306102e5b067d08a29094f37b8193_4.0")
]

,progress_id,student_id,course_id,module_number,lesson_number,status,start_date,finish_date,course_title,course_field,module_title,lesson_title,lesson_token,is_video,is_homework,course-module_id
334902,d0cecd82f603e457eb1142def0fa3104,47c3c0e20c441a4fa021ca4bd8044e27,943306102e5b067d08a29094f37b8193,4.0,27.0,done,2019-04-11 10:56:03.684644,2019-04-14 03:12:23.012366,Java-разработчик c нуля,Development,"Числа, строки и даты",Домашняя работа 4.6,ed373787-d237-4fb3-8248-82595b0dde56,False,True,943306102e5b067d08a29094f37b8193_4.0
335494,d0cecd82f603e457eb1142def0fa3104,47c3c0e20c441a4fa021ca4bd8044e27,943306102e5b067d08a29094f37b8193,4.0,21.0,done,2019-04-10 10:10:39.013230,2019-04-11 07:12:20.545344,Java-разработчик c нуля,Development,"Числа, строки и даты",Домашняя работа 4.4,e817fd9a-5417-42fa-b7c7-5eb4b073a4c5,False,True,943306102e5b067d08a29094f37b8193_4.0
336535,d0cecd82f603e457eb1142def0fa3104,47c3c0e20c441a4fa021ca4bd8044e27,943306102e5b067d08a29094f37b8193,4.0,13.0,done,2019-04-08 11:21:12.288867,2019-04-09 12:48:04.289564,Java-разработчик c нуля,Development,"Числа, строки и даты",Домашняя работа 4.2,3979e3be-0215-4752-881b-0580bc6d2e0f,False,True,943306102e5b067d08a29094f37b8193_4.0
337489,d0cecd82f603e457eb1142def0fa3104,47c3c0e20c441a4fa021ca4bd8044e27,943306102e5b067d08a29094f37b8193,4.0,17.0,done,2019-04-10 09:53:14.194003,2019-04-11 07:08:48.389104,Java-разработчик c нуля,Development,"Числа, строки и даты",Домашняя работа 4.3,6179c00f-24d9-438b-a2c8-063e6974eb79,False,True,943306102e5b067d08a29094f37b8193_4.0
339267,d0cecd82f603e457eb1142def0fa3104,47c3c0e20c441a4fa021ca4bd8044e27,943306102e5b067d08a29094f37b8193,4.0,6.0,done,2019-04-08 09:55:35.476712,2019-04-09 12:08:28.996480,Java-разработчик c нуля,Development,"Числа, строки и даты",Домашняя работа 4.1,69816d53-c480-44cf-9fd1-3f4ec66a71c3,False,True,943306102e5b067d08a29094f37b8193_4.0
339981,d0cecd82f603e457eb1142def0fa3104,47c3c0e20c441a4fa021ca4bd8044e27,943306102e5b067d08a29094f37b8193,4.0,25.0,done,2019-04-11 10:12:13.605879,2019-04-13 12:40:59.080275,Java-разработчик c нуля,Development,"Числа, строки и даты",Домашняя работа 4.5,cd9b006e-25ac-424d-b33c-5f1e56190172,False,True,943306102e5b067d08a29094f37b8193_4.0


Вывод: прогресс считается общим для модуля (!придумать лучшее описание)

Lessons in module completed by one student share common progress_id

Need to just group by course-module_id and then by progress_id

In [20]:
for module_id_, module_gr_ in dataset[dataset["is_homework"] == True].groupby("course-module_id"):
    
    min_series = module_gr_.groupby("progress_id")["start_date"].min()
    max_series = module_gr_.groupby("progress_id")["finish_date"].max()
    duration_series = max_series - min_series
    break

duration_series

progress_id
0029682bdb5be81e3bc3966ab728cf0c   20 days 07:37:28.844132
005313709cc6b84861f660eb8d9fe507    1 days 16:11:31.274623
00ac441da5e8bc71c5677e435f785704    0 days 10:01:34.978087
014fb5de0f52f4cf59addf261a4219ff    0 days 01:34:47.750355
019debce205c4bfc0e4d7a9557badbee    0 days 09:13:21.243457
                                             ...          
fb6d26ecb84a4085f10274033ac9c44d    0 days 17:37:21.567894
fc0d4061f40019bbc0099cc7b1f8f0fa    0 days 02:34:20.198109
fc6cc45a85034ffc32eac4f94d228262    4 days 11:09:14.902012
fcd863a85271512bc5df54875a2d3005    1 days 08:35:25.229628
feb5ee78d496dceaf87645e4413ac399    4 days 23:07:25.323323
Length: 342, dtype: timedelta64[ns]

In [11]:
data_agg = pd.DataFrame()

for module_id_, module_gr_ in dataset[dataset["is_homework"] == True].groupby("course-module_id"):
    
    if module_gr_["lesson_number"].nunique() == 1:
        
        average_time = (module_gr_["finish_date"] - module_gr_["start_date"]).mean()
        median_time = (module_gr_["finish_date"] - module_gr_["start_date"]).median()
        
        temp_dict = {
        "course-module_id" : module_id_,
        "course_title" : module_gr_["course_title"].iloc[0],
        "module_number" : module_gr_["module_number"].iloc[0],
        "average_time" : average_time,
        "median_time" : median_time
        }
        data_agg = data_agg.append(temp_dict, ignore_index=True)
        
    else:
        
        first_hw_num = module_gr_["lesson_number"].min()
        last_hw_num = module_gr_["lesson_number"].max()
        
        #print(module_id_, first_hw_num, last_hw_num, module_gr_["student_id"].nunique())
        
data_agg.set_index("course-module_id", inplace=True)
data_agg

/home/iamcomputer/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


,course_title,module_number,average_time,median_time
course-module_id,,,,
04ba6d0b40670c43a209141fa01fa784_1.0,Анимация интерфейсов,1.0,11 days 19:41:17.659012238,1 days 22:31:56.639415
04ba6d0b40670c43a209141fa01fa784_10.0,Анимация интерфейсов,10.0,13 days 09:09:49.877145857,6 days 01:01:22.889799500
04ba6d0b40670c43a209141fa01fa784_11.0,Анимация интерфейсов,11.0,26 days 07:41:06.099055628,12 days 23:33:59.876374
04ba6d0b40670c43a209141fa01fa784_12.0,Анимация интерфейсов,12.0,NaT,NaT
04ba6d0b40670c43a209141fa01fa784_13.0,Анимация интерфейсов,13.0,24 days 15:00:26.129520963,9 days 17:02:13.213271
...,...,...,...,...
e9bb9205eeed307ee7cbaa08bfd166c3_5.0,JavaScript с нуля,5.0,17 days 10:51:14.284782232,10 days 00:24:29.275534
e9bb9205eeed307ee7cbaa08bfd166c3_6.0,JavaScript с нуля,6.0,16 days 07:50:56.554498042,8 days 13:22:11.486400
e9bb9205eeed307ee7cbaa08bfd166c3_7.0,JavaScript с нуля,7.0,20 days 16:23:08.435682182,11 days 05:01:06.596879


### Time to complete module - maximum

### Time to complete module - average

### Time to complete module - median